In [1]:
%matplotlib inline
# python libraties
import os, cv2,itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image


# pytorch libraries
import torch
from torch import optim,nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms

# sklearn libraries
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# to make the results are reproducible
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

#import plotly.express as px
import pandas as pd

print(os.listdir("../input"))

['skin-cancer-mnist-ham10000']


Counting the benign images

In [2]:
pd.read_csv("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv").groupby("dx")["dx"].count()

dx
akiec     327
bcc       514
bkl      1099
df        115
mel      1113
nv       6705
vasc      142
Name: dx, dtype: int64

In [3]:
#df = pd.read_csv("/kaggle/input/siim-isic-melanoma-classification/train.csv")
df2 = pd.read_csv("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv")
lesion_type_dict = {
    'nv': 'nevus',
    'mel': 'melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

# Use map to replace short forms with long forms
df2['diagnosis'] = df2['dx'].map(lesion_type_dict)

In [4]:
lesion_type_dict = {
    'nv': 'nevus',
    'mel': 'melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

# Use map to replace short forms with long forms
benign_mal['diagnosis'] = benign_mal_2['dx'].map(lesion_type_dict)

NameError: name 'benign_mal_2' is not defined

In [5]:
data_dir = '/kaggle/input/skin-cancer-mnist-ham10000/'
all_image_path = glob(os.path.join(data_dir, '*', '*.jpg'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'dermatofibroma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [6]:
all_image_path2 = glob(os.path.join(data_dir))

In [7]:
all_image_path2

['/kaggle/input/skin-cancer-mnist-ham10000/']

In [8]:
def compute_img_mean_std(image_paths):
    """
        computing the mean and std of three channel on the whole dataset,
        first we should normalize the image from 0-255 to 0-1
    """

    img_h, img_w = 224, 224
    imgs = []
    means, stdevs = [], []

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        imgs.append(img)

    imgs = np.stack(imgs, axis=3)
    print(imgs.shape)

    imgs = imgs.astype(np.float32) / 255.

    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  # resize to one row
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))

    means.reverse()  # BGR --> RGB
    stdevs.reverse()

    print("normMean = {}".format(means))
    print("normStd = {}".format(stdevs))
    return means,stdevs

In [9]:
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

In [19]:
dict(zip(df_original["cell_type_idx"], df_original["cell_type"]))

{2: 'Benign keratosis-like lesions ',
 4: 'Melanocytic nevi',
 3: 'Dermatofibroma',
 6: 'dermatofibroma',
 5: 'Vascular lesions',
 1: 'Basal cell carcinoma',
 0: 'Actinic keratoses'}

In [10]:
df_original = pd.read_csv(os.path.join('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv'))
df_original['path'] = df_original['image_id'].map(imageid_path_dict.get)
df_original['cell_type'] = df_original['dx'].map(lesion_type_dict.get)
df_original['cell_type_idx'] = pd.Categorical(df_original['cell_type']).codes
df_original.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/kaggle/input/skin-cancer-mnist-ham10000/ham10...,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/kaggle/input/skin-cancer-mnist-ham10000/ham10...,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/kaggle/input/skin-cancer-mnist-ham10000/ham10...,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/kaggle/input/skin-cancer-mnist-ham10000/ham10...,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/kaggle/input/skin-cancer-mnist-ham10000/ham10...,Benign keratosis-like lesions,2


In [17]:
df_original.groupby(["cell_type", "cell_type_idx"]).value_counts()

cell_type          cell_type_idx  lesion_id    image_id      dx     dx_type  age   sex     localization     path                                                                            
Actinic keratoses  0              HAM_0000005  ISIC_0024579  akiec  histo    75.0  female  lower extremity  /kaggle/input/skin-cancer-mnist-ham10000/ham10000_images_part_1/ISIC_0024579.jpg    1
                                               ISIC_0025577  akiec  histo    75.0  female  lower extremity  /kaggle/input/skin-cancer-mnist-ham10000/ham10000_images_part_1/ISIC_0025577.jpg    1
                                               ISIC_0029638  akiec  histo    75.0  female  lower extremity  /kaggle/input/skin-cancer-mnist-ham10000/ham10000_images_part_2/ISIC_0029638.jpg    1
                                               ISIC_0030591  akiec  histo    75.0  female  lower extremity  /kaggle/input/skin-cancer-mnist-ham10000/ham10000_images_part_2/ISIC_0030591.jpg    1
                                  H

In [46]:
# this will tell us how many images are associated with each lesion_id
df_undup = df_original.groupby('lesion_id').count()
# now we filter out lesion_id's that have only one image associated with it
df_undup = df_undup[df_undup['image_id'] == 1]
df_undup.reset_index(inplace=True)
df_undup.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
0,HAM_0000001,1,1,1,1,1,1,1,1,1
1,HAM_0000003,1,1,1,1,1,1,1,1,1
2,HAM_0000004,1,1,1,1,1,1,1,1,1
3,HAM_0000007,1,1,1,1,1,1,1,1,1
4,HAM_0000008,1,1,1,1,1,1,1,1,1


In [47]:
# here we identify lesion_id's that have duplicate images and those that have only one image.
def get_duplicates(x):
    unique_list = list(df_undup['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'

# create a new colum that is a copy of the lesion_id column
df_original['duplicates'] = df_original['lesion_id']
# apply the function to this new column
df_original['duplicates'] = df_original['duplicates'].apply(get_duplicates)
df_original.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx,duplicates
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/kaggle/input/skin-cancer-mnist-ham10000/ham10...,Benign keratosis-like lesions,2,duplicated
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/kaggle/input/skin-cancer-mnist-ham10000/ham10...,Benign keratosis-like lesions,2,duplicated
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/kaggle/input/skin-cancer-mnist-ham10000/ham10...,Benign keratosis-like lesions,2,duplicated
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/kaggle/input/skin-cancer-mnist-ham10000/ham10...,Benign keratosis-like lesions,2,duplicated
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/kaggle/input/skin-cancer-mnist-ham10000/ham10...,Benign keratosis-like lesions,2,duplicated


In [48]:
df_original['duplicates'].value_counts()

duplicates
unduplicated    5514
duplicated      4501
Name: count, dtype: int64

In [49]:
# now we filter out images that don't have duplicates
df_undup = df_original[df_original['duplicates'] == 'unduplicated']
df_undup.shape

(5514, 11)

In [50]:
# now we create a val set using df because we are sure that none of these images have augmented duplicates in the train set
y = df_undup['cell_type_idx']
_, df_val = train_test_split(df_undup, test_size=0.2, random_state=101, stratify=y)
df_val.shape

(1103, 11)

In [51]:
df_val['cell_type_idx'].value_counts()

cell_type_idx
4    883
2     88
6     46
1     35
0     30
5     13
3      8
Name: count, dtype: int64

In [52]:
# This set will be df_original excluding all rows that are in the val set
# This function identifies if an image is part of the train or val set.
def get_val_rows(x):
    # create a list of all the lesion_id's in the val set
    val_list = list(df_val['image_id'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# identify train and val rows
# create a new colum that is a copy of the image_id column
df_original['train_or_val'] = df_original['image_id']
# apply the function to this new column
df_original['train_or_val'] = df_original['train_or_val'].apply(get_val_rows)
# filter out train rows
df_train = df_original[df_original['train_or_val'] == 'train']
print(len(df_train))
print(len(df_val))

8912
1103


In [53]:
print(df_train['cell_type_idx'].value_counts())
print(df_val['cell_type'].value_counts())

cell_type_idx
4    5822
6    1067
2    1011
1     479
0     297
5     129
3     107
Name: count, dtype: int64
cell_type
Melanocytic nevi                  883
Benign keratosis-like lesions      88
dermatofibroma                     46
Basal cell carcinoma               35
Actinic keratoses                  30
Vascular lesions                   13
Dermatofibroma                      8
Name: count, dtype: int64


In [54]:
# Copy fewer class to balance the number of 7 classes
data_aug_rate = [15, 10, 5, 50, 0, 40, 5]
augmented_parts = [df_train]  # Start with original

for i in range(7):
    if data_aug_rate[i] > 1:
        extra_copies = [df_train.loc[df_train['cell_type_idx'] == i, :]] * (data_aug_rate[i] - 1)
        augmented_parts.extend(extra_copies)

df_train = pd.concat(augmented_parts, ignore_index=True)
print(df_train['cell_type'].value_counts())

cell_type
Melanocytic nevi                  5822
Dermatofibroma                    5350
dermatofibroma                    5335
Vascular lesions                  5160
Benign keratosis-like lesions     5055
Basal cell carcinoma              4790
Actinic keratoses                 4455
Name: count, dtype: int64


In [55]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35967 entries, 0 to 35966
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   lesion_id      35967 non-null  object 
 1   image_id       35967 non-null  object 
 2   dx             35967 non-null  object 
 3   dx_type        35967 non-null  object 
 4   age            35880 non-null  float64
 5   sex            35967 non-null  object 
 6   localization   35967 non-null  object 
 7   path           35967 non-null  object 
 8   cell_type      35967 non-null  object 
 9   cell_type_idx  35967 non-null  int8   
 10  duplicates     35967 non-null  object 
 11  train_or_val   35967 non-null  object 
dtypes: float64(1), int8(1), object(10)
memory usage: 3.1+ MB


In [56]:
# # We can split the test set again in a validation set and a true test set:
# df_val, df_test = train_test_split(df_val, test_size=0.5)
df_train = df_train.reset_index()
df_val = df_val.reset_index()
# df_test = df_test.reset_index()

In [57]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [58]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0

    if model_name == "densenet":
        """ Densenet121 """
        model_ft = models.densenet121(pretrained=use_pretrained)
        # Freeze parameters if feature_extract is True
        if feature_extract:
            for param in model_ft.parameters():
                param.requires_grad = False
        # Replace classifier
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        raise ValueError(f"Unsupported model: {model_name}")

    return model_ft, input_size

model_name = 'densenet'
num_classes = 7
feature_extract = False
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Define the device:
model = torch.nn.DataParallel(model_ft)  # Wrap in DataParallel
model = model.to('cuda')   

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [59]:
print("Using", torch.cuda.device_count(), "GPUs")

Using 2 GPUs


In [60]:
# norm_mean = (0.49139968, 0.48215827, 0.44653124)
# norm_std = (0.24703233, 0.24348505, 0.26158768)
# define the transformation of the train images.
train_transform = transforms.Compose([transforms.Resize((input_size,input_size)),transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
                                        transforms.ToTensor(), transforms.Normalize(norm_mean, norm_std)])
# define the transformation of the val images.
val_transform = transforms.Compose([transforms.Resize((input_size,input_size)), transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

In [61]:
# Define a pytorch dataloader for this dataset
class HAM10000(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load data and get label
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            X = self.transform(X)

        return X, y

In [62]:
# Define the training set using the table train_df and using our defined transitions (train_transform)
training_set = HAM10000(df_train, transform=train_transform)
train_loader = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=4)
# Same for the validation set:
validation_set = HAM10000(df_val, transform=train_transform)
val_loader = DataLoader(validation_set, batch_size=32, shuffle=False, num_workers=4)

In [63]:
# we use Adam optimizer, use cross entropy loss as our loss function
device = torch.device('cuda')
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)

In [64]:
# this function is used during training process, to calculation the loss and accuracy
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [65]:
total_loss_train, total_acc_train = [],[]
def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    for i, data in enumerate(train_loader):
        images, labels = data
        N = images.size(0)
        # print('image shape:',images.size(0), 'label shape',labels.size(0))
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 100 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    return train_loss.avg, train_acc.avg

In [66]:
def validate(val_loader, model, criterion, optimizer, epoch):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    return val_loss.avg, val_acc.avg

In [67]:
epoch_num = 10
best_val_acc = 0
total_loss_val, total_acc_val = [],[]
for epoch in range(1, epoch_num+1):
    loss_train, acc_train = train(train_loader, model, criterion, optimizer, epoch)
    loss_val, acc_val = validate(val_loader, model, criterion, optimizer, epoch)
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        print('*****************************************************')
        print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')

[epoch 1], [iter 100 / 1124], [train loss 1.33314], [train acc 0.50406]
[epoch 1], [iter 200 / 1124], [train loss 1.22120], [train acc 0.55266]
[epoch 1], [iter 300 / 1124], [train loss 1.12880], [train acc 0.58448]
[epoch 1], [iter 400 / 1124], [train loss 1.07437], [train acc 0.59930]
[epoch 1], [iter 500 / 1124], [train loss 1.03655], [train acc 0.61381]
[epoch 1], [iter 600 / 1124], [train loss 0.99728], [train acc 0.62792]
[epoch 1], [iter 700 / 1124], [train loss 0.96160], [train acc 0.63964]
[epoch 1], [iter 800 / 1124], [train loss 0.93656], [train acc 0.64879]
[epoch 1], [iter 900 / 1124], [train loss 0.91107], [train acc 0.65924]
[epoch 1], [iter 1000 / 1124], [train loss 0.88996], [train acc 0.66734]
[epoch 1], [iter 1100 / 1124], [train loss 0.86706], [train acc 0.67568]
------------------------------------------------------------
[epoch 1], [val loss 0.41825], [val acc 0.84887]
------------------------------------------------------------
***********************************

In [68]:
model_cpu = model.module if isinstance(model, torch.nn.DataParallel) else model
model_cpu = model_cpu.to('cpu')

# Now save the weights from CPU
torch.save(model_cpu.state_dict(), '/kaggle/working/best_model_weights.pt')

In [47]:
print(df_train['path'].isnull().sum())
print(df_train[df_train['path'].isnull()])

35967
       index    lesion_id      image_id   dx dx_type   age     sex  \
0          0  HAM_0000118  ISIC_0027419  bkl   histo  80.0    male   
1          1  HAM_0000118  ISIC_0025030  bkl   histo  80.0    male   
2          2  HAM_0002730  ISIC_0026769  bkl   histo  80.0    male   
3          3  HAM_0002730  ISIC_0025661  bkl   histo  80.0    male   
4          4  HAM_0001466  ISIC_0031633  bkl   histo  75.0    male   
...      ...          ...           ...  ...     ...   ...     ...   
35962  35962  HAM_0001953  ISIC_0025611  mel   histo  65.0    male   
35963  35963  HAM_0002552  ISIC_0032985  mel   histo  25.0    male   
35964  35964  HAM_0002552  ISIC_0032936  mel   histo  25.0    male   
35965  35965  HAM_0002552  ISIC_0033232  mel   histo  25.0    male   
35966  35966  HAM_0003521  ISIC_0032258  mel   histo  70.0  female   

          localization  path                       cell_type  cell_type_idx  \
0                scalp  None  Benign keratosis-like lesions               

In [48]:
df_train = df_train[df_train['path'].notnull()].reset_index(drop=True)

In [92]:
training_set

In [93]:
df_train

,index,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx,duplicates,train_or_val
0,0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,None,Benign keratosis-like lesions,2,duplicated,train
1,1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,None,Benign keratosis-like lesions,2,duplicated,train
2,2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,None,Benign keratosis-like lesions,2,duplicated,train
3,3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,None,Benign keratosis-like lesions,2,duplicated,train
4,4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,None,Benign keratosis-like lesions,2,duplicated,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,35962,HAM_0001953,ISIC_0025611,mel,histo,65.0,male,back,None,dermatofibroma,6,duplicated,train
35963,35963,HAM_0002552,ISIC_0032985,mel,histo,25.0,male,upper extremity,None,dermatofibroma,6,duplicated,train
35964,35964,HAM_0002552,ISIC_0032936,mel,histo,25.0,male,upper extremity,None,dermatofibroma,6,duplicated,train
35965,35965,HAM_0002552,ISIC_0033232,mel,histo,25.0,male,upper extremity,None,dermatofibroma,6,duplicated,train
